In [12]:
# -*- coding: utf-8 -*-
#!/usr/bin/python

import RPi.GPIO as GPIO
import os
import numpy as np
import smbus
import time
import signal
import pickle

class ADXL345():
    def __init__(self):
        
        self.DevAdr = 0x53
        myBus = ""
        if GPIO.RPI_INFO['P1_REVISION'] == 1:
            myBus = 0
        else:
            myBus = 1
            
        self.b = smbus.SMBus(myBus)
        
        self.b.write_byte_data(self.DevAdr, 0x2C, 0x0B) # BandwidthRate　originally 0B 
        self.b.write_byte_data(self.DevAdr, 0x31, 0x00) # DATA_FORMAT 10bit 2g
        self.b.write_byte_data(self.DevAdr, 0x38, 0x00) # FIFO_CTL OFF
        self.b.write_byte_data(self.DevAdr, 0x2D, 0x08) # POWER_CTL Enable
    
    def getOffset(self):
        data =[]
        for a in range(NO_OF_OFFSET_MEASURE):
            input = self.getData()
            data.append(input)
            time.sleep(SAMPLING_CYCLE)
        data = np.array(data)
        offset = data.mean(axis = 0)
        
        return offset

    def getData(self):
        output = []
        data = self.b.read_i2c_block_data(self.DevAdr, 0x32, 6)
        for i in range(0,6,2):
            sign = data[i + 1] & 0x80
            tmp = data[i + 1] & 0x7F
            tmp = tmp << 8
            tmp = tmp | data[i]

            if sign > 0:
                output.append(tmp - 32768)
            else:
                output.append(tmp)

        return np.array(output)

def main(arg1, arg2):
    
    global Data
    
    print "*",
    
    ema = np.zeros(3)
    data = myADXL345.getData()
    data = np.array(data - offset)
    ema = (1 - FORGETING_FACTOR) * ema + FORGETING_FACTOR * data
    Data.append(ema)
    print "*",

    if time.time() - TIME_START >= MONITOR_TIME:
        signal.setitimer(signal.ITIMER_REAL, 0.)
        print "\nDone"
    
        with open('Data.pkl', 'wb') as f:
            pickle.dump(Data, f)
            
if __name__ == "__main__":
    
    SAMPLING_CYCLE = 0.01 # in seconds
    MONITOR_TIME = 20 # in seconds
    
    NO_OF_SAMPLES = int(MONITOR_TIME * ( 1.0  / SAMPLING_CYCLE ))
    
    OFFSET_MEASURE_TIME = 0.5
    NO_OF_OFFSET_MEASURE = int(OFFSET_MEASURE_TIME * ( 1.0  / SAMPLING_CYCLE ))
    
    FORGETING_FACTOR = 0.1
    
    myADXL345 = ADXL345()

    offset = myADXL345.getOffset()

    Data =[]
    
    TIME_START = time.time()
    
    signal.signal(signal.SIGALRM, main)
    signal.setitimer(signal.ITIMER_REAL, 0.1, SAMPLING_CYCLE)

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

In [13]:
from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
output_notebook()

with open('Data.pkl', 'rb') as f:
    Data = pickle.load(f)

Data = list(map(list, zip(*Data)))  #Swap rows and columns

t=[float(n) * SAMPLING_CYCLE for n in range(0,NO_OF_SAMPLES)]

pp = figure(tools='xwheel_zoom,xpan',
title="",
x_axis_label='Time[sec]',
y_axis_label='Acceleration')
pp.line(t, Data[0],legend="X", line_width=1, line_color = "blue")
pp.line(t, Data[1],legend="Y", line_width=1, line_color = "red")
pp.line(t, Data[2],legend="Z", line_width=1, line_color = "green")
output_file("robot.html")
show(pp)

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 2000), ('y', 1991)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python2.7/dist-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 2000), ('y', 1991)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python2.7/dist-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 2000), ('y', 1991)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
